In [1]:
""" Filter India data

aws emr add-steps --cluster-id <Your EMR cluster id> --steps Type=spark,Name=TestJob,Args=[--deploy-mode,cluster,--master,yarn,--conf,spark.yarn.submit.waitAppCompletion=true,s3a://your-source-bucket/code/pythonjob.py,s3a://your-source-bucket/data/data.csv,s3a://your-destination-bucket/test-output/],ActionOnFailure=CONTINUE
"""

from collections import namedtuple
import logging
import sys

from geopy.distance import great_circle
import pandas as pd
import geopandas as gpd
import numpy as np

from datetime import timedelta, date, datetime
from statistics import *
from shapely.geometry import MultiPolygon, Polygon
from shapely import wkt


from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import (
    StructType,
    LongType,
    StructField,
    IntegerType,
    StringType,
    DoubleType,
    TimestampType,
    ArrayType
)
import pyspark.sql.functions as F
from math import *
import time
import json


spark = SparkSession.builder.appName(f"trips_veraset").getOrCreate()
spark.sparkContext.addPyFile("s3://ipsos-dvd/scripts/utils.py")

fn = "s3://ipsos-dvd/fdd/data/pings_IN_2021-09-01_2021-09-30/"




VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1689255436350_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
india = spark.read.parquet(fn)
india.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+-------------+--------+
|                caid| latitude|longitude|utc_timestamp|altitude|
+--------------------+---------+---------+-------------+--------+
|0c076b4217a5f8bee...|26.897369|75.772568|   1631088670|    null|
|0c076b4217a5f8bee...|26.897369|75.772568|   1631089868|    null|
|757d9b3c494ff6fa6...| 17.37527| 78.47443|   1631203951|    null|
|757d9b3c494ff6fa6...| 17.37528| 78.47444|   1631203962|     0.0|
|fd5a8877a234a3bfa...| 28.49767| 77.15836|   1630979266|    null|
|fd5a8877a234a3bfa...| 28.49767| 77.15836|   1630979299|    null|
|fd5a8877a234a3bfa...|28.497683|77.158348|   1630979309|    null|
|fd5a8877a234a3bfa...|28.497705|77.158348|   1630979340|    null|
|fd5a8877a234a3bfa...|  28.4977| 77.15836|   1630979395|    null|
|fd5a8877a234a3bfa...| 28.49767| 77.15836|   1630979629|    null|
|fd5a8877a234a3bfa...| 28.49178|  77.1588|   1631008465|    null|
|fd5a8877a234a3bfa...| 28.49178|  77.1588|   1631008499|    null|
|fd5a8877a

In [8]:
# filter on bounding box
minx, maxx = 68.716667, 80.983333 # Longitude
miny, maxy = 22.166667, 30.916667 # Latitude

# filter on rajastan
india = india.filter(F.col('longitude').between(minx, maxx) & F.col('latitude').between(miny, maxy))

# filter on internet blackout dates
india = india.withColumn("date", F.from_utc_timestamp(F.col("utc_timestamp").cast(TimestampType()), tz = "IST"))
india = india.filter(F.to_date(F.col("date")).isin(["2021-09-05", "2021-09-12", "2021-09-19", "2021-09-26"]))

india.write.mode("overwrite").parquet("s3://ipsos-dvd/fdd/data/rajasatan_sundays_sep_2021")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+-------------+--------+-------------------+
|                caid| latitude|longitude|utc_timestamp|altitude|               date|
+--------------------+---------+---------+-------------+--------+-------------------+
|453f3255b987c9c96...| 22.28827| 73.25702|   1631449573|    null|2021-09-12 17:56:13|
|453f3255b987c9c96...| 22.28827| 73.25702|   1631449606|    null|2021-09-12 17:56:46|
|453f3255b987c9c96...|22.288288|73.237543|   1631461094|     0.0|2021-09-12 21:08:14|
|453f3255b987c9c96...|22.288288|73.237543|   1631461279|     0.0|2021-09-12 21:11:19|
|453f3255b987c9c96...|22.288288|73.237543|   1631461459|     0.0|2021-09-12 21:14:19|
|40165ec6d23ec641b...| 22.95495| 73.33912|   1631434634|    null|2021-09-12 13:47:14|
|40165ec6d23ec641b...| 22.95496| 73.33914|   1631434643|    null|2021-09-12 13:47:23|
|40165ec6d23ec641b...| 22.95496| 73.33914|   1631436658|    null|2021-09-12 14:20:58|
|40165ec6d23ec641b...|22.846846| 73.33935|   163144062